# Measuring the Certainty of the k-Nearest Neighbors Algorithm developed in Python for Visualizing the 3D Stratigraphic Architecture of the Llobregat River Delta

In [1]:
DATADIR='data/' # Directory with the data
FIGURESDIR='figures/' # Figures produced

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import os

import shapely.geometry as geometry

from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors._base import _get_weights

In [4]:
Heights=list(range(0,-101,-5)) # Heights where the certainty maps will be obtained

FRAME=500 # Plot frame

In [5]:
knn_grav = NearestNeighbors(n_neighbors=1)
knn_sand = NearestNeighbors(n_neighbors=1)
knn_clay = NearestNeighbors(n_neighbors=1)
knn_base = NearestNeighbors(n_neighbors=4)

def certainty(X):
    neight_grav=knn_grav.kneighbors(X)[0].reshape((len(X)))
    neight_sand=knn_sand.kneighbors(X)[0].reshape((len(X)))
    neight_clay=knn_clay.kneighbors(X)[0].reshape((len(X)))
    
    m = min(knn_base.n_samples_fit_, 4)
    neight_base=knn_base.kneighbors(X, m)[0].reshape((len(X),m))
        
    cert=np.zeros(len(X))    
        
    for i in range(len(X)):
        bas=sorted(list(neight_sue[i]))
        no_bas=[neight_grav[i],neight_sand[i],neight_clay[i]]
        m_no_bas=min(no_bas)
        w=_get_weights(np.array([no_bas+[bas[0]]]), 'distance')[0]
        cert[i]=np.max(w)/np.sum(w)
        
        s=1
        while s<m and bas[s]<m_no_bas:
            w=_get_weights(np.array([no_bas+[bas[s]]]), 'distance')[0]
            cert[i]=cert[i]+(1-cert[i])*(w[3]-max(w[:3]))/np.sum(w)
            s+=1
        
    return 100*cert

In [6]:
cmap = plt.cm.rainbow
cmaplist = [cmap(i) for i in range(cmap.N)]
cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)
bounds=np.array([25, 30, 35, 40, 50, 60, 70, 80, 90, 95, 100])
norm=mpl.colors.BoundaryNorm(bounds, cmap.N)

In [6]:
data=pd.read_excel(DATADIR+'hsd new basements.xls')
data=data.drop(columns=['Codi'])

ax=[data.UTM_X.min()-FRAME,data.UTM_X.max()+FRAME,data.UTM_Y.min()-FRAME,data.UTM_Y.max()+FRAME]
minx_rounded = 1000 * round(ax[0]/1000)
maxx_rounded = 1000 * round(ax[1]/1000)

xx, yy = np.meshgrid(np.linspace(ax[0],ax[1],300),np.linspace(ax[2],ax[3],300))
C=np.zeros(xx.shape,dtype = float)

contorno=pd.read_csv('cont2c7.csv')
contorno=contorno.drop(columns=['dentro', 'hoja', 'Cota'])
poligono=geometry.Polygon(zip(contorno['UTMX'],contorno['UTMY']))
#poligono = alpha_shape(list(zip(X,Y)), alpha=ALPHA, default_distance=DEFAULT_DISTANCE)[0].buffer(BUFFER)  

outside=[] # Puntos exteriores al polígono
for i in range(xx.shape[0]):
    for j in range(xx.shape[1]):
        if poligono.distance(geometry.Point(xx[i,j],yy[i,j]))>1:
            outside.append((i,j))
            
xpol,ypol = poligono.exterior.xy

for COTA in COTAs:
    print('COTA:',COTA)

    df=datos[datos.Cota==COTA] # Me quedo con las de cierta Cota  
        
    gravillas=df[df.Clase=='gravillas y gravas']
    arenas=df[df.Clase=='arenas']
    arcillas=df[df.Clase=='arcillas y limos']
    suelos=df[df.Valor=='S']
    
    knn_grav.fit(list(zip(gravillas['UTM_X'],gravillas['UTM_Y'])))
    knn_are.fit(list(zip(arenas['UTM_X'],arenas['UTM_Y'])))
    knn_arc.fit(list(zip(arcillas['UTM_X'],arcillas['UTM_Y'])))
    knn_sue.fit(list(zip(suelos['UTM_X'],suelos['UTM_Y'])))
        
    for i in range(xx.shape[0]):
        d=list(zip(xx[i],yy[i]))
        C[i]=reliability(d)

    for (i,j) in outside:
        C[i,j]=np.nan

    plt.imshow(C, vmin = 25, vmax = 100, cmap=cmap, origin='lower', norm=norm, alpha=0.6,
           extent=[xx.min(), xx.max(), yy.min(), yy.max()])
    
    plt.colorbar(spacing='proportional').set_label(' %', rotation=0)
    
    plt.scatter(gravillas['UTM_X'], gravillas['UTM_Y'], c='blue', s=1, alpha=0.9, label='gravillas')
    plt.scatter(arenas['UTM_X'], arenas['UTM_Y'], c='orange', s=1, alpha=0.8, label='arenas')
    plt.scatter(arcillas['UTM_X'], arcillas['UTM_Y'], c='black', s=1, alpha=0.7, label='arcillas')
    plt.scatter(suelos['UTM_X'], suelos['UTM_Y'], c='brown', s=1, alpha=0.7, label='suelos')

    plt.plot(xpol, ypol, alpha=0.6, color='red', linewidth=1.5, linestyle='dashed')    

    plt.xlabel('UTM_X')
    plt.ylabel('UTM_Y')

    plt.xticks(np.arange(minx_rounded, maxx_rounded, step=3000))
    
    plt.title("Height "+str(COTA)+' m')
    plt.axis(ax)

    filename='cota'+str(COTA)+'.png'
    
    filename='reliability/'+filename
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    plt.savefig(filename, transparent=False, dpi=164, bbox_inches='tight')

    plt.clf()
    #plt.show()

COTA: 0
COTA: -5
COTA: -10
COTA: -15
COTA: -20
COTA: -25
COTA: -30
COTA: -40
COTA: -50
COTA: -60
COTA: -80
COTA: -100


<Figure size 432x288 with 0 Axes>